# Trabajo práctico N° 4
## Mellino, Natalia

In [ ]:
## Imports
# install.packages("adabag")
library("adabag")

## Ejercicio 1

In [3]:
## Ejemplo
data("iris")
train <- c(sample(1:50, 25), sample(51:100, 25), sample(101:150, 25))
iris.adaboost <- boosting(Species ~ ., data=iris[train, ], mfinal=10, coef="Freund", control=rpart.control(maxdepth=3))
iris.adaboost

$formula
Species ~ .

$trees
$trees[[1]]
n= 75 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

1) root 75 41 setosa (0.4533333 0.3600000 0.1866667)  
  2) Petal.Length< 2.45 34  0 setosa (1.0000000 0.0000000 0.0000000) *
  3) Petal.Length>=2.45 41 14 versicolor (0.0000000 0.6585366 0.3414634)  
    6) Petal.Length< 4.75 27  0 versicolor (0.0000000 1.0000000 0.0000000) *
    7) Petal.Length>=4.75 14  0 virginica (0.0000000 0.0000000 1.0000000) *

$trees[[2]]
n= 75 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

1) root 75 26 versicolor (0.2000000 0.6533333 0.1466667)  
  2) Petal.Length< 2.7 15  0 setosa (1.0000000 0.0000000 0.0000000) *
  3) Petal.Length>=2.7 60 11 versicolor (0.0000000 0.8166667 0.1833333)  
    6) Petal.Width< 1.75 49  0 versicolor (0.0000000 1.0000000 0.0000000) *
    7) Petal.Width>=1.75 11  0 virginica (0.0000000 0.0000000 1.0000000) *

$trees[[3]]
n= 75 

node), split, n, loss, yval, (yprob)
      * denotes terminal nod

## Ejercicio 2

## Ejercicio 3

## Ejercicio 4